# Channel equalization

![title](profile/group02_profile.jpg)

## Introduction

In wireless communication. The wireless channels consist of various types of impairments such as delay spread, fading and Doppler spread, etc. Multipath propagation in channel introduces delay spread that cause ISI, ICI,....


In order to encouter that, we need to do equalization to reduce the distortion, mitigate the combined effect of ISI and noise. Thus to restore the original signal, we used a filter, there are two basic filterL Zero Forcing or MMSE (Minimum Mean Square Error)

## Objective 

In this note book, we will show how the Zero Forcing equalizer and MMSE equalizer work mathematically, and how to calculate it
![title](theory_pics/2-Figure1-1.png)

## Zero forcing equalizer

With Zero Forcing qualizaer we have 
$$
    z(m) = C(m)*h(m) \\
     (m = -N,....+N)\\  
     z(m)=\sum \limits _{n=-N} ^{n=+N} C(n).h(m-n)
$$
Where 
       z : Ouput of the filter C : Filter vector h: Input signal


In matrix
$$
\vec z = h. \vec c\\
\vec c = h^{-1} . \vec z
$$


In this part. we will take an example from the book, and calculate the filter output based on the equations, the evaluate the output

Before begining the calculation we need to import the numpy libary which contains most of the functions we are going to use

In [ ]:
import numpy as np
from numpy.linalg import inv

Base on the above equation, we first need to create the impulse respone matrix from the impusle respone of the channel

In [ ]:
# this function generate a matrix base on a input array, offset is based on the input array
def generate_square_matrix(arr_data,size,data_offset,datatype):
    aMatrix = np.mat(np.zeros(shape=(size,size))).astype(datatype)
    for i1 in range(size):    
        for i2 in range(size):
            try:
                arr_index = i2+data_offset-i1
                if arr_index < 0:
                    continue
                aMatrix[i2,i1]=arr_data[arr_index]
            except:
                break
    return aMatrix

As the example in the book we have the respon value of
$$
h(-2)= 0.5 h(-11) = -0.8 h(0) = 1.4 h(1) = 0.7 h(2) = 0.1 h(3) = 0.2
$$
##### With the length of 5

In [ ]:
arr = np.array([ 0.5, -0.8, 1.4, 0.7, -0.6, 0.2])

In [ ]:
# For this case we use offset is the index of the largest value of the impusle 
# since h(0) the has the highest value
ZF_h = generate_square_matrix(arr,5,arr.argmax(),float)
print("impulse respone matrix:\n" ,ZF_h)

After we got the impulse matrix we need to invert it as above equation

In [ ]:
ZF_h_inv = inv(ZF_h) #invert the impulse respone matrix
np.set_printoptions(formatter={'float': lambda ZF_h_inv: "{0:0.4f}".format(ZF_h_inv)})
print("Inverted impulse respone matrix: \n",ZF_h_inv)

Then the Vector C of the filter is the middle column of the inverted impulse respone matrix, we take the vector

In [ ]:
# This is the funtion which take a matrix and extract the middle column of the matrix to a array
def get_C_vector(inv_matrix, size,data_type):
    mid_col=size>>1
    C_vector=np.zeros(shape=(size)).astype(data_type)
    for i in range(size):
        C_vector[i]= inv_matrix[i,mid_col]
    return C_vector

ZF_C_vec = get_C_vector(ZF_h_inv,5,float)
print("The C vector: ",ZF_C_vec)

At this point, we got the C vector of the filter, since this example doesn't have input signal, we will see how the filter work by convolve it with the original impulse respone array

In [ ]:
# Recheck with convolution of  C vector and h(t)
def evaluate_ZF(ZF_C_vec,arr):
    return np.convolve(ZF_C_vec, arr)
ZF_z=evaluate_ZF(ZF_C_vec,arr)
print(ZF_z)

Since the filter has the length of 5, we have to ignore the first N = 2 elemment (where filter length = 2 N + 1), and we only care about the 3th to the 8th elements

In [ ]:
print(ZF_z[2:7])

#### This is the proper result, we can see that at the point z[0] value is 1 and other elements are zeros

Another example from the book:

In [ ]:
def ZF_equalizer(y,h,size,size_of_input,data_type):
    ZF_h = generate_square_matrix(h,size,h.argmax(),data_type)
    ZF_h_inv = inv(ZF_h)
    np.set_printoptions(formatter={'float': lambda ZF_h_inv: "{0:0.4f}".format(ZF_h_inv)})
    print("\nImpulse respone matrix:\n" ,ZF_h,"\n\nInverted impulse respone matrix: \n",ZF_h_inv)
    ZF_C_vec = get_C_vector(ZF_h_inv,5,data_type)
    print("\nThe C vector: ",ZF_C_vec)
    z=np.convolve(y,ZF_C_vec)
    leftside=size>>1
    print("\nReconstructed signal: \n",z[leftside:leftside+size_of_input])    
    return z[leftside:leftside+size_of_input]

x=np.array([1+1j,-1+3j,-3-3j,-1-3j])    
# y = np.array([1+1j,-1.8+2.2j,-1.7-4.9j,0.7+0.7j,-0.5+0.3j,0.1-0.9j,0.2+0.6j])
h = np.array([ 1, -0.8, 0.5, -0.2])
y = np.convolve(x,h)
print("\nRneceived signal :", y) 
ZF_result = ZF_equalizer(y,h,5,x.size,complex)
print("\nOriginal signal:\n",x)

In [ ]:
# np.square(np.subtract(x, ZF_result)).mean()

### Conclusion
    This menthod does not consider the interference and the FIR filter can only mitigate the minimum cased of ISI interference where distortion smaller than 100% ISI eye. 

## MMSE equalizer

MMSE equation:
$$
    e^2(N)=E[(z(n)-x(n))^2]\\ 
$$
with
$$
z(n)= C(n)*y(n)=\sum\limits _{m=-N} ^{+N} C(m).y(n-m)\\
     (m = -N,....+N)
$$

in matrix form
$$
R_{vv} \vec C = \vec R_{xy}\\
\vec C = R_{yy}^{-1} \vec R_{yy}
$$

With MMSE we also follow the example in the note book with input of x_m and output is y_m with filter length is 5

In [ ]:
x_m=np.array([0.4501,-0.2689,0.1068,0])
y_m=np.array([0.3601,-0.3601,0.1661,-0.0321])
size = 5
data_type = float

First we need to find correlation of y and cross correlation of x

In [ ]:
ryy=np.correlate(y_m,y_m,"full")
rxy=np.correlate(x_m,y_m,"full")
print('ryy:',ryy,'\n\nrxy',rxy,'\n')

Then we generate matrix Ryy from ryy and vector Rxy form rxy

In [ ]:
Ryy=generate_square_matrix(ryy,size,ryy.argmax(),data_type)
Rxy = np.mat(np.zeros(shape=(size,1))).astype(data_type)
offset=rxy.argmax() - (size>>1)
for i in range(size):
    Rxy[i,0]= rxy[i+offset]
print("Ryy: \n",Ryy,"\n\nRxy: \n",Rxy)

The we got the C vector of the filter

In [ ]:
MMSE_C_Vec= np.asarray(inv(Ryy)*Rxy).flatten()
print(MMSE_C_Vec)

Then the result, similar with ZF filter we have to get to get rid of some irrelevent value in the result

In [ ]:
result= np.convolve(y_m,MMSE_C_Vec)
np.set_printoptions(formatter={data_type: lambda result: "{0:0.4f}".format(result)})
leftside=size>>1
print(result[leftside:leftside+x.size])
print("\nOriginal :",x_m)

In [ ]:
np.square(np.subtract(x_m, result[leftside:leftside+x.size])).mean()

Another example from the book:

In [ ]:
def MMSE_equalizer(x,y,size,data_type):
    ryy=np.correlate(y,y,"full")
    rxy=np.correlate(x,y,"full")
    print('ryy:',ryy,'\n\nrxy',rxy,'\n')

    Ryy=generate_square_matrix(ryy,size,ryy.argmax(),data_type)
    Rxy = np.mat(np.zeros(shape=(size,1))).astype(data_type)
    offset=rxy.argmax() - (size>>1)
    for i in range(size):
            Rxy[i,0]= rxy[i+offset]
    np.set_printoptions(formatter={data_type: lambda result: "{0:0.4f}".format(Ryy)})
    print("Ryy: \n",Ryy,"\n\nRxy: \n",Rxy)
    MMSE_C_Vec= np.asarray(inv(Ryy)*Rxy).flatten()
    print("\nVector C:\n",MMSE_C_Vec)
    result = np.convolve(y,MMSE_C_Vec)

    np.set_printoptions(formatter={data_type: lambda result: "{0:0.4f}".format(result)})
    leftside=size>>1
    print("\nRecontructed signal:\n",result[leftside:leftside+x.size])
    print("\n Original: ",x,"\nReceived signal:",y)
    mse1=np.square(np.subtract(x, result[leftside:leftside+x.size])).mean()
    mse2=np.square(np.subtract(x, y)).mean()

    print("\nMean Square error with recontructed signal:\n",mse1)
    print("\nMean Square error with recieved signal:\n",mse2)
    print("\nDifferent:",x-result[leftside:leftside+x.size])
    print("\n",result)
z=np.array([1+1j,-1+3j,-3-3j,-1-3j])
w=np.array([1+1j,-1.8+2.2j,-1.7-4.9j,0.7+0.7j,-0.5+0.3j,0.1-0.9j,0.2+0.6j])
MMSE_equalizer(z,w,5,complex)


### Conclusion
    This menthod does have adavantages over Zero Forcing shortcoming and also it's easy to use, easy to calculate, and very versatile

In [ ]:
from pyphysim.util.misc import pretty_time, randn_c
from pyphysim.util.conversion import dB2Linear

# z=np.array([1+1j,-1+3j,-3-3j,-1-3j,2+4j,1+1j,-1+3j,-3-3j,-1-3j,2+4j,1+1j,-1+3j,-3-3j,-1-3j,2+4j,1+1j,-1+3j,-3-3j,-1-3j,2+4j,1+1j,-1+3j,-3-3j,-1-3j,2+4j,1+1j,-1+3j,-3-3j,-1-3j,2+4j,1+1j,-1+3j,-3-3j,-1-3j,2+4j,1+1j,-1+3j,-3-3j,-1-3j,2+4j])
z=np.array([0.73+0.59j  ,0.43+1.01j  ,0.41+0.3j   ,1.24+1.1j  , 0.55+0.83j ])
SNR_dB = 30
snr_linear = dB2Linear(SNR_dB)
noise_power = 1 / snr_linear
# Noise vector
n = math.sqrt(noise_power) * randn_c(z.size)
# print(modulated_data_qpsk.size)

h = randn_c(z.size)
# Receive the corrupted data
y_z = h * z + n
# Equalization
y_z /= h
print(y_z)
MMSE_equalizer(z,y_z,5,complex)